R# Preprocesamiento de los comentarios para la primera fase de fine-tuning y entrenamiento del modelo
En esta primera fase, una vez extraídos los bodies de todos los issues/PRs de los repositorios listados junto con 2 comentarios de 250 del total de issues de cada repositorio, se prepararán todos los datos para ser utilizados en el entrenamiento y ajuste fino del modelo que se utilizará.

Se seguirán prácticamente los mismos pasos vistos en *c_preparing_data_for_statistics_and_ML* pero con varias diferencias claves que existen entre los modelos BERT que se utilizarán ahora y los modelos de clasificación presentados con anterioridad (notebooks de GVTIA).

Para Transformers funciona mejor un preprocesado mínimo y dejar la segmentación al propio tokenizador del modelo, a continuación se muestra que procedimientos similares a los anteriores se mantendrán y cuáles se evitarán:

## Se mantendrá:
- Normalización de espacios/saltos de línea
- Eliminación de caracteres de control raros o poco usuales
- Se conservará el uso de mayúsculas y minúsculas, signos, números, URLs, nombres propios de vulnerabilidades o bugs (CVE-XXXX-YYYY), rutas (/etc/...), código entre backticks (`return salida`), nombres de APIs.
- Se definirá una longitud máxima de tokens por comentario o el uso de *sliding window* si el texto es muy largo.

## Se omitirá:
- Pasar todo el texto a minúsculas, los modelos RoBERTa/DistilRoBERTa que se utilizarán utilizan mayúsculas y minúsculas.
- Eliminar la puntuación y stopwords.
- Stemming / lematización.
- Normalizaciones agresivas de URLs/código -> se pierde señal técnica.

Una vez explicado esto, se comenzará con el preprocesado de todos los comentarios extraídos de GitHub, comenzando como se ha visto ya en diversas ocasiones, con cargar el documento (.csv) en un dataframe de pandas para su uso y manipulación.

En este caso, se cuenta con 2 documentos:
- **gh_bodys_lastyear.csv**. Archivo que contiene los bodies (comentario principal) de todos los Issues/PRs en el último año de los repositorios listados para la extracción de comentarios.
- **gh_comments_lastyear.csv**. Archivo que contiene los 2 primeros comentarios de cada Issue/PR de 250 Issues/PRs por repositorio (500 comentarios por repo), en gran parte de los casos serán las respuestas aportadas por usuarios al body del documento anterior.

En este caso, como se cuenta con 2 documentos lo que se hará es crear 2 dataframes, uno con cada documento, para a continuación unirlos con la función `concat()` de pandas y ordenarlos según el id del Issue/PR para la clara visualización y mantener una estructura coherente entre cuerpo principal y comentarios asociados.

In [1]:
import os
import subprocess
# Cargamos el archivo settings_bert.py que contiene todos los imports necesarios para la correcta ejecución del notebook y no tener que importarlos en cada celda de código
%run -i ../settings_bert.py

In [ ]:
# Ruta de los archivos
path_gh_bodys = "../data/gh_comments/training/gh_bodys_lastyear.csv"
path_gh_comments = "../data/gh_comments/training/gh_comments_lastyear.csv"

# Carga de los archivos en DataFrames
df_bodys = pd.read_csv(path_gh_bodys)
df_comms = pd.read_csv(path_gh_comments)

In [3]:
print(df_bodys.columns)
print(df_comms.columns)

Index(['repo', 'is_pr', 'issue_number', 'comment_type', 'comment_id',
       'comment_created_at', 'comment_author', 'text', 'comment_url',
       'context_id', 'container_title', 'container_state', 'container_url',
       'container_created_at', 'container_updated_at', 'container_labels'],
      dtype='object')
Index(['kubernetes/kubernetes', 'False', '133680', 'issue_comment',
       'github_issuecomment_IC_kwDOAToIks6_4TOW', '2025-08-25T07:51:17Z',
       'k8s-ci-robot',
       'This issue is currently awaiting triage.\nIf a SIG or subproject determines this is a relevant issue, they will accept it by applying the triage/accepted label and provide further guidance.\nThe triage/accepted label can be added by org members by writing /triage accepted in a comment.\n\nInstructions for interacting with me using PR comments are available here.  If you have questions or suggestions related to my behavior, please file an issue against the kubernetes-sigs/prow repository.',
       'https://gi

Se ha cometido un error en la escritura de la cabecera de los comentarios por escribir siempre en el mismo documento y borrar su contenido en vez de eliminar el documento antes de comenzar con una nueva extracción. Vamos a tratar de repararlo sin tener que volver a realizar todo el proceso de extracción.

In [4]:
path_gh_bodys = Path(path_gh_bodys)
path_gh_comments = Path(path_gh_comments)

EXPECTED_COLS = [
    'repo','is_pr','issue_number','comment_type','comment_id','comment_created_at','comment_author',
    'text','comment_url','context_id','container_title','container_state','container_url',
    'container_created_at','container_updated_at','container_labels'
]

def read_with_header_fix(p: Path) -> pd.DataFrame:
    # Se lee 1 fila para inspeccionar columnas
    probe = pd.read_csv(p, nrows=1)
    if list(probe.columns) == EXPECTED_COLS:
        return pd.read_csv(p)
    # Si no coincide, reinterpretamos: no hay cabecera -> header=None + names=EXPECTED_COLS
    return pd.read_csv(p, header=None, names=EXPECTED_COLS)

df_bodys = read_with_header_fix(path_gh_bodys)
df_comms = read_with_header_fix(path_gh_comments)

# Se unen ambos DataFrames
df_gh = pd.concat([df_bodys, df_comms], ignore_index=True)

# Tipos y ordenación
df_gh['comment_created_at'] = pd.to_datetime(df_gh['comment_created_at'], errors='coerce', utc=True)
df_gh.loc[df_gh['comment_created_at'].isna(), 'comment_created_at'] = pd.to_datetime(df_gh['container_created_at'], errors='coerce', utc=True)

order_map = {'issue_body':0, 'pr_body':0} # Bodies primero -> coherencia
df_gh['order'] = df_gh['comment_type'].map(order_map).fillna(1).astype(int)

df_gh = df_gh.sort_values(by=['repo','issue_number','order','comment_created_at','comment_id'], kind='mergesort').drop(columns=['order'])

# Normalizar booleano -> OPCIONAL
df_gh['is_pr'] = df_gh['is_pr'].astype(str).str.lower().map({'true':True, 'false':False})

In [5]:
# Muestra para comprobar que se ha ejecutado correctamente
print(df_bodys.columns)
print(df_comms.columns)

df_gh.head(10).T
print(len(df_gh))

Index(['repo', 'is_pr', 'issue_number', 'comment_type', 'comment_id',
       'comment_created_at', 'comment_author', 'text', 'comment_url',
       'context_id', 'container_title', 'container_state', 'container_url',
       'container_created_at', 'container_updated_at', 'container_labels'],
      dtype='object')
Index(['repo', 'is_pr', 'issue_number', 'comment_type', 'comment_id',
       'comment_created_at', 'comment_author', 'text', 'comment_url',
       'context_id', 'container_title', 'container_state', 'container_url',
       'container_created_at', 'container_updated_at', 'container_labels'],
      dtype='object')
108078


Ahora sí están todos los comentarios bien ordenados. Antes de comenzar con el preprocesado vamos a guardar el dataframe en una base de datos.

In [6]:
# Compruebo que todos los comentarios se han almacenado correctamente en el DF
print(len(df_gh))

108078


In [7]:
db_gh = "../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db"
con = sqlite3.connect(db_gh)
df_gh.to_sql('gh_dataset', con, if_exists='replace', index=False)
con.close()

Ahora sí se procederá al procesamiento del dataset para dejarlo preparado para el modelo BERT que se utilizará, RoBERTa o DistilRoBERTa. Este proceso se va a definir en una serie de scripts .py, cada uno con el objetivo de realizar una tarea para su reutilización en otros puntos del proyecto (cuando se haga el de reddit, u otros comentarios de github) de forma que estos sean agnósticos al sistema del que se extraen los comentarios que serán utilizados por el modelo.

Del mismo modo, tras el procesamiento de los datos, el resultado del procesado será almacenado en archivos `.parquet` por su ligereza y agilidad a la hora de ser manipulados y consumidos por modelos BERT. Las principales ventajas de este formato son:
- Más pequeño: compresión por columna, pesa de 2 a 5 veces menos que un `.csv`
- Más rápido: lee solo las columnas que se necesitan ("column pruning") y aplica vectorización.
- Conserva tipos: fechas, booleanos, enteros "nullable", etc. (`.csv`los pierde)
- Esquema: guarda el _schema_ dentro del archivo -> menos sorpresas al cargarlo

Por estas características el formato es el preferido para pipelines de datos/ML. En este caso la estructura que se utilizará será:
- `merged.parquet` = será el dataset completo tras ingesta y normalización ligera.
- `split_train.parquet`, `split_dev.parquet`, `split_test.parquet` = particiones del dataset ya divididas, listas para su tokenización.

#### Ventajas frente a CSV
- Evita problemas de comas y saltos de línea
- Mantiene las fechas (`created_at`), booleanos (`is_pr`) y enteros sin perder el tipo.
- Carga solo las columnas necesarias -> menor uso de RAM y tiempo de ejecución.

## Ejecución del preprocesado de datos
Se han definido varios scripts, cada uno con una función en el preprocesado de datos para modelos BERT, a  los cuáles se les llamará desde este notebook con los argumentos correspondientes para realizar este proceso.

Las funciones reutilizables para el pipeline están definidas en `prep_utils.py`. Normaliza texto "ligero", mapea columnas heterogéneas (GH/Reddit) al esquema core (id, text, label, source, created_at, context_id), lee CSV/SQLite y guarda Parquet.
El objetivo principal de estas funciones es su utilización cuando se desea agnosticismo de fuente (Github/Reddit) y un preprocesado mínimo ideal para Transformers.

### `ingest_merge.py`
Combina una o varias entradas en un único DataFrame, deduplica el contenido por `id`, aplica normalización ligera y guarda `merged.parquet` + meta. Se le da como input el archivo `.db` o los `.csv` deseados, produciendo en la salida un DF en formato `.parquet`.

In [8]:
base = "../src/data_prep"

# 1. Ingesta + merge (de CSVs o db en SQLite)
# Como ya tengo el dataset directamente almacenado en un .db
subprocess.run([sys.executable, f"{base}/ingest_merge.py",
                "--sqlite-db", "../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db",
                "--table", "gh_dataset",
                "--out-parquet", "../src/artifacts/prep/merged.parquet",
                "--out-meta", "../src/artifacts/prep/merged_meta.json"
                ], check=True)


CompletedProcess(args=['C:\\Users\\diego\\AppData\\Local\\Programs\\Python\\Python310\\python.exe', '../src/data_prep/ingest_merge.py', '--sqlite-db', '../data/gh_comments/train-fine_tuning/gh_dataset_lastyear.db', '--table', 'gh_dataset', '--out-parquet', '../src/artifacts/prep/merged.parquet', '--out-meta', '../src/artifacts/prep/merged_meta.json'], returncode=0)

### `quick_report.py`
Muestra por pantalla un resumen del Parquet aportado en el input: nº de filas, distribución de `label`, distribución de `source`, rango de `created_at`.
Es útil para la verificación visual de que la ingesta es correcta antes del split/tokenización e ideal para detectar desbalanceos fuertes o rangos temporales inesperados antes de la fase de entrenamiento.

In [9]:
# Informe rápido
subprocess.run([sys.executable, f"{base}/quick_report.py",
                "--in-parquet", "../src/artifacts/prep/merged.parquet"
                ], check=True)

CompletedProcess(args=['C:\\Users\\diego\\AppData\\Local\\Programs\\Python\\Python310\\python.exe', '../src/data_prep/quick_report.py', '--in-parquet', '../src/artifacts/prep/merged.parquet'], returncode=0)

### `split_thread_temporal.py`
Divide el DF `merged.parquet` en train/dev/test respetando hilos (`context_id`) y orden temporal para evitar fuga de información entre splits manteniendo el orden cronológico dentro de cada uno.
Los `ratios` son las proporciones con las que se dividirá el dataset:
- train: 70%
- dev: 15%
- test: 15%

In [10]:
# Split temporal + thread-aware
subprocess.run([sys.executable, f"{base}/split_thread_temporal.py",
                "--in-parquet", "../src/artifacts/prep/merged.parquet",
                "--out-train", "../src/artifacts/prep/split_train.parquet",
                "--out-dev", "../src/artifacts/prep/split_dev.parquet",
                "--out-test", "../src/artifacts/prep/split_test.parquet",
                "--ratios", "0.7", "0.15", "0.15",
                "--out-meta", "../src/artifacts/prep/split_meta.json"
                ], check=True)

CompletedProcess(args=['C:\\Users\\diego\\AppData\\Local\\Programs\\Python\\Python310\\python.exe', '../src/data_prep/split_thread_temporal.py', '--in-parquet', '../src/artifacts/prep/merged.parquet', '--out-train', '../src/artifacts/prep/split_train.parquet', '--out-dev', '../src/artifacts/prep/split_dev.parquet', '--out-test', '../src/artifacts/prep/split_test.parquet', '--ratios', '0.7', '0.15', '0.15', '--out-meta', '../src/artifacts/prep/split_meta.json'], returncode=0)

### `tokenize_hf.py`
Carga los Parquet de train/dev/test, antepone un prefijo de dominio (`<GITHUB>`, `<REDDIT>`, en este caso no porque todavía estoy solo con los comentarios de GH), tokeniza los textos con el tokenizador de DistilRoBERTa/RoBERTa y guarda:
- `dataset/` (formato `save_to_disk` de HF Datasets)
- `tokenizer/` (vocabulario + config)
- `preprocess_meta.json` (tamaños, `max_len`, pesos de cada clase)

#### Parámetros
- `--base-model`: modelo que se utilizará, en este caso distilroberta
- `--max-len 384`: longitud de secuencia; bajar acelera, subir captura más contexto.
- `--use-domain-prefix`: etiqueta de la fuente (GITHUB/REDDIT)
- `--sliding-window --slide-stride 128`: para textos muy largos (menos truncado)

Para decidir el `--max-len` y la posibilidad de utilizar `--sliding-window`, se puede calcular la distribución de longitudes en tokens y elegir el valor del argumento para cubrir el *p90-p95* y obtener un mejor resultado.

In [11]:
tok = AutoTokenizer.from_pretrained("distilroberta-base", use_fast=True)
df = pd.read_parquet("../src/artifacts/prep/merged.parquet") # o split_train.parquet
lens = df["text"].astype(str).map(lambda s: len(tok(s, truncation=False)["input_ids"]))

print(pd.Series(lens).describe(percentiles=[.5, .9, .95, .99]))

Token indices sequence length is longer than the specified maximum sequence length for this model (523 > 512). Running this sequence through the model will result in indexing errors


count   108000.00
mean       339.97
std       1068.08
min          3.00
50%         89.00
90%        666.00
95%       1380.05
99%       4944.00
max      54918.00
Name: text, dtype: float64


### Argumentos del tokenizador
- `--max-len 384`. Como el modelo solo puede leer hasta N tokens al mismo tiempo, se limita a N tokens (384) en la tokenización de cada texto. Si este tiene un nº superior se trunca o se divide en trozos (sliding-window). Elijo 384 tokens máx. por texto por su equilibrio en velocidad y uso de VRAM (no debería de tener problema con mi PC).
- `--sliding-window`. Si un texto no cabe en `--max-len`, en lugar de descartar el sobrante se divide en ventanas solapadas, dando como resultado varias filas por cada texto largo.
- `--slide-stride 128`. Es el número de tokens que se solapan entre ventanas para no cortar frases/palabras a la mitad cuando se divide un texto entre ventanas. En este caso si el texto se divide en 2 ventanas la distribución sería:
    - Ventana 1: tokens 1-384
    - Ventana 2: tokens (384-128) 256-639
- `--max-windows-per-doc 6`. Limita a un máximo de 6 ventanas por texto, el resto se recorta. Es mejor que descartar el texto completo, y si supera el máx. de ventanas es muy probable que sea debido a mensajes de logs o diffs y no aporte nada al modelo, solo ruido.
- `--filter-max-input-tokens`. Como se ve en la celda anterior (porcentajes) hay textos enormes y con este argumento se descartan sus colas limitando a N tokens antes de dividir en ventanas.
- `--num-proc 12`. Usa 12 hilos de ejecución para map paralelo (Mi PC tiene 8C/16T) -> Implementar en el tokenizador
- `--batch-chunk-size 512`. Indica el tamaño de trozos (grandes) en cada llamada al tokenizador. -> Implementar en el tokenizador

#### Recomendaciones
- Si ves que sube mucho el nº de ejemplos → baja --max-windows-per-doc a 4.
- Si va justo de RAM → --num-proc 8 y/o --batch-chunk-size 256.

In [12]:
env = dict(os.environ)

# Fuerza UTF-8 en el hijo:
env["PYTHONIOENCODING"] = "utf-8"
env["PYTHONUTF8"] = "1"

# Paralelización
env["TOKENIZERS_PARALLELISM"] = "false" #evita sub-proceso
env["RAYON_NUM_THREADS"] = "2" #2 hilos por proceso

# tokenización HF (DistilRoBERTa/RoBERTa)
res = subprocess.run([sys.executable, f"{base}/tokenize_hf.py",
                "--train-parquet", "../src/artifacts/prep/split_train.parquet",
                "--dev-parquet", "../src/artifacts/prep/split_dev.parquet",
                "--test-parquet", "../src/artifacts/prep/split_test.parquet",
                "--out-dir", "../src/artifacts/hf_distilroberta",
                "--base-model", "distilroberta-base",
                "--max-len", "384",
                "--sliding-window",
                "--slide-stride", "128",
                "--max-window-per-doc", "8",
                "--filter-max-input-tokens", "8192",
                #"--use-domain-prefix", # Se activa cuando incluya Reddit
                "--num-proc", "8",
                "--batch-chunk-size", "512"
                ], check=True, env=env, capture_output=True, text=True, encoding="utf-8", errors="replace")

print(res.stdout)   # logs del script
print(res.stderr)   # avisos/tracebacks del script

[filtro] train -> 263 descartados por > 8192 tokens
[filtro] validation -> 36 descartados por > 8192 tokens
[filtro] test -> 37 descartados por > 8192 tokens
OK -> ..\src\artifacts\hf_distilroberta\dataset ..\src\artifacts\hf_distilroberta\tokenizer

Token indices sequence length is longer than the specified maximum sequence length for this model (848 > 384). Running this sequence through the model will result in indexing errors

Map (num_proc=8): 100%|██████████| 73841/73841 [00:21<00:00, 3363.27 examples/s]

Map (num_proc=8): 100%|██████████| 16293/16293 [00:11<00:00, 1398.61 examples/s]

Map (num_proc=8): 100%|██████████| 17530/17530 [00:11<00:00, 1497.80 examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 114631/114631 [00:00<00:00, 906325.01 examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 23877/23877 [00:00<00:00, 852746.46 examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 24396/24396 [00:00<00:00, 871282.09 examples/s]



**Factor de expansión** -> Si se va demasiado, bajar --max-window a 4

In [ ]:
len(tok_train)/len(ds_train)

**Sanity check** -> Comprobar que no hay basura

In [ ]:
tokenizer.decode(tok_train["input_ids"][0])[:300]

**Velocidad** -> Si el tiempo de ejecución es muy elevado usar --max-len 256 o subir --slide-stride a 192 para menos ventanas